In [1]:
!pip install pytorch_lightning
!pip install torch_lr_finder
!pip install lightning
!pip install grad-cam
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
import torch.optim as optim
import lightning.pytorch as pl
from lightning.pytorch.tuner import Tuner

# import pytorch_lightning as pl
from tqdm import tqdm

from torch.optim.lr_scheduler import OneCycleLR
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import albumentations as A
import cv2
import torch
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
from albumentations.pytorch import ToTensorV2


In [5]:
from google.colab import drive

drive.mount('/content/gdrive')
import shutil
drive_path = '/content/gdrive/My Drive/yolov3_model_without_75_mosaic.pth'
local_path = 'yolov3_model_without_75_mosaic.pth'  # The path where you saved the model within Colab
shutil.copyfile(drive_path, local_path)

Mounted at /content/gdrive


In [2]:
from utils_for_app import cells_to_bboxes,non_max_suppression,plot_image,YoloCAM
from yolov3 import YOLOv3
from loss import YoloLoss
from utils import LearningRateFinder
# Create your config module or import it from the existing config.py file.
import config
from main_yolov3_lightening import YOLOv3Lightning
import torch
import cv2
import numpy as np
import gradio as gr

In [3]:
model = YOLOv3Lightning()
model.load_state_dict(torch.load("yolov3_model_without_75_mosaic.pth", map_location=torch.device('cpu')), strict=False)
model.setup(stage="test")

In [4]:
IMAGE_SIZE = 416
transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=IMAGE_SIZE),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE, min_width=IMAGE_SIZE, border_mode=cv2.BORDER_CONSTANT
        ),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
        ToTensorV2(),
    ],
)

ANCHORS = [
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)],
]  # Note these have been rescaled to be between [0, 1]
S = [IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8]


In [5]:
scaled_anchors = (
    torch.tensor(config.ANCHORS)
    * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
)

def process_image_and_plot(image, model, scaled_anchors):

    transformed_image = transforms(image=image)["image"].unsqueeze(0)
    output = model(transformed_image)
    bboxes = [[] for _ in range(1)]

    for i in range(3):
        batch_size, A, S, _, _ = output[i].shape
        anchor = scaled_anchors[i]
        boxes_scale_i = cells_to_bboxes(output[i], anchor, S=S, is_preds=True)
        for idx, box in enumerate(boxes_scale_i):
            bboxes[idx] += box

    nms_boxes = non_max_suppression(
        bboxes[0], iou_threshold=0.5, threshold=0.4, box_format="midpoint",
    )
    fig = plot_image(transformed_image[0].permute(1, 2, 0), nms_boxes)

    cam = YoloCAM(model=model, target_layers=[model.model.layers[-2]], use_cuda=False)
    grayscale_cam = cam(transformed_image, scaled_anchors)[0, :, :]
    img = cv2.resize(image, (416, 416))
    img = np.float32(img) / 255
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)

    return fig,cam_image


In [7]:
import gradio as gr
examples = [
    ["/content/images/automobile.jpg"],
    ["/content/images/cycle.jpg"],
    ["/content/images/dog-kitten.jpg"],
    ["/content/images/human.jpg"],
]

def processed_image(image):
    figure,gradcam = process_image_and_plot(image, model, scaled_anchors)
    return figure,gradcam

title = "YoloV3 on Pascal VOC Dataset with GradCAM"
description = "Pytorch Lightening Implemetation of YoloV3 trained from scratch"
demo = gr.Interface(processed_image,
    inputs=[
        gr.Image(label="Input Image"),
    ],
    outputs=[gr.Plot(),gr.Image(shape=(32, 32), label="Model Prediction")],
    title=title,
    description=description,
    examples=examples,
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>